# Homework 3

## Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_url = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
df = pd.read_csv(csv_url)
print(df.head())

    lead_source    industry  number_of_courses_viewed  annual_income  \
0      paid_ads         NaN                         1        79450.0   
1  social_media      retail                         1        46992.0   
2        events  healthcare                         5        78796.0   
3      paid_ads      retail                         2        83843.0   
4      referral   education                         3        85012.0   

  employment_status       location  interaction_count  lead_score  converted  
0        unemployed  south_america                  4        0.94          1  
1          employed  south_america                  1        0.80          0  
2        unemployed      australia                  3        0.69          1  
3               NaN      australia                  1        0.87          0  
4     self_employed         europe                  3        0.62          1  


## Data preparation
- Check if the missing values are presented in the features.
- If there are missing values:
    - For categorical features, replace them with 'NA'
    - For numerical features, replace with with 0.0

In [4]:
missing_values_across_df = df.isnull().sum(); missing_values_across_df

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
# lead_source = categorical
# industry = categorical
# number_of_courses_viewed = numerical
# annual_income = numerical
# employment_status = categotical
# location = categorical
# interaction_count = numerical
# lead_score = numerical
# converted = categorical - TARGET

In [6]:
categorical_missing = ['lead_source', 'industry', 'employment_status', 'location']
numerical_missing = ['annual_income']

In [7]:
# Fill missing values
df[categorical_missing] = df[categorical_missing].fillna('NA')
df[numerical_missing] = df[numerical_missing].fillna(0.0)

In [8]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [9]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Question 1
What is the most frequent observation (mode) for the column industry?
- `NA`
- `technology`
- `healthcare`
- `retail`

In [10]:
most_frequent = df['industry'].mode()[0]; most_frequent

'retail'

In [11]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

## Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.
<br>
What are the two features that have the biggest correlation?
- `interaction_count` and `lead_score`
- `number_of_courses_viewed` and `lead_score`
- `number_of_courses_viewed` and `interaction_count`
- `annual_income` and `interaction_count`
<br>
Only consider the pairs above when answering this question.

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [14]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [15]:
# Drop the indecies
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [16]:
# y variables
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [17]:
# delete the target variable
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
numerical = ['number_of_courses_viewed','annual_income','interaction_count', 'lead_score']

In [19]:
categorical = [
 'lead_source',
 'industry',
 'employment_status',
 'location']

In [20]:
df_full_train[numerical].corrwith(df_full_train.converted)

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

In [21]:
pair1 = ['interaction_count', 'lead_score']
pair2 = ['number_of_courses_viewed','lead_score']
pair3 = ['number_of_courses_viewed','interaction_count']
pair4 = ['annual_income','interaction_count']

In [22]:
df_full_train[pair1].corr().abs()

,interaction_count,lead_score
interaction_count,1.000000,0.025393
lead_score,0.025393,1.000000


In [23]:
df_full_train[pair2].corr().abs()

,number_of_courses_viewed,lead_score
number_of_courses_viewed,1.000000,0.009427
lead_score,0.009427,1.000000


In [24]:
df_full_train[pair3].corr().abs()

,number_of_courses_viewed,interaction_count
number_of_courses_viewed,1.000000,0.044381
interaction_count,0.044381,1.000000


In [25]:
df_full_train[pair4].corr().abs()

,annual_income,interaction_count
annual_income,1.000000,0.011959
interaction_count,0.011959,1.000000


In [26]:
# Third pair, ['number_of_courses_viewed','interaction_count'], has the highest absolute value

## Split the data
- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value converted is not in your dataframe.

## Question 3
- Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

- `industry`
- `location`
- `lead_source`
- `employment_status`

In [27]:
from sklearn.metrics import mutual_info_score

In [28]:
categorical = [
 'lead_source',
 'industry',
 'employment_status',
 'location']

In [29]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [30]:
mi = df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending = False)
mi

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

`lead_source` has the highest mutual information

## Question 4
- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.


In [31]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

Let's see how the full model looks like

In [32]:
dicts_full_train = df_full_train[categorical +  numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
round((churn_decision == y_test).mean(),2)

0.74

## Question 5
- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.


In [33]:
dicts_full_train = df_full_train[categorical +  numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
original_accuracy = (churn_decision == y_test).mean(); original_accuracy

0.7372013651877133

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.

In [34]:
# no industry
categorical_1 = ['lead_source', 'employment_status', 'location']

In [35]:
# no employment_status
categorical_2 = ['lead_source', 'industry', 'location']

In [36]:
# no lead_score
numerical_1 = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count']

In [37]:
dicts_full_train = df_full_train[categorical_1 +  numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
first_accuracy = (churn_decision == y_test).mean(); first_accuracy

0.7372013651877133

In [38]:
dicts_full_train = df_full_train[categorical_2 +  numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
second_accuracy = (churn_decision == y_test).mean(); second_accuracy

0.726962457337884

In [39]:
dicts_full_train = df_full_train[categorical +  numerical_1].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
third_accuracy = (churn_decision == y_test).mean(); third_accuracy

0.7406143344709898

Which of following feature has the smallest difference?

- `industry`
- `employment_status`
- `lead_score`

Note: The difference doesn't have to be positive.

In [40]:
#Difference:
abs(original_accuracy-first_accuracy)

0.0

In [41]:
abs(original_accuracy-second_accuracy)

0.010238907849829282

In [42]:
abs(original_accuracy-third_accuracy)

0.0034129692832765013

industry yields no difference in accuracy

## Question 6
- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.


In [43]:
dicts_full_train = df_full_train[categorical +  numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)
y_full_train = df_full_train.converted.values

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
dicts_test = df_test[categorical +  numerical].to_dict(orient = 'records')
X_test = dv.transform(dicts_test)
y_pred = model.predict_proba(X_test)[:, 1]
churn_decision = (y_pred >= 0.5)
original_accuracy = (churn_decision == y_test).mean(); original_accuracy

0.7372013651877133

In [44]:
dicts_train = df_train[categorical + numerical].to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(dicts_train)
# y_train is in the beginning

reg_parameters = [0.01, 0.1, 1, 10, 100]
for c in reg_parameters:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42).fit(X_full_train, y_full_train)
    dicts_val = df_val[categorical +  numerical].to_dict(orient = 'records')
    X_val = dv.transform(dicts_val)
    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)
    original_accuracy = (churn_decision == y_val).mean()
    print(round(original_accuracy,3))

0.706
0.717
0.717
0.717
0.717


0.1 is the smallest; the rest of the C regularization parameters result in the same accuracy.